In [ ]:
from common import *
import holoviews as hv

use_hvplot()

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

In [ ]:
PLOT_OPTS = dict(
    width=1200, 
    height=600
)
year = 2021
water_year = f'wy{year}'

In [ ]:
smeshr_path = f'/uufs/chpc.utah.edu/common/home/u1135435/skiles-group1/SMESHR/Shade/{water_year}/'
smrf_path = f'/uufs/chpc.utah.edu/common/home/u1135435/skiles-group1/SMESHR/Shade_SMRF/{water_year}/'

In [ ]:
smeshr_solar = xr.open_mfdataset(
    smeshr_path + '*.nc',
    parallel=True
)
# Get rid of sunrise and sunset time entries by rounding them down
# Enables comparison to full hour SMRF values
smeshr_solar['time'] = smeshr_solar.time.dt.floor('1H')

In [ ]:
# Create index that excludes the daylight savings time change
_data, index = np.unique(smeshr_solar.azimuth.time, return_index=True)

In [ ]:
smrf_solar = xr.open_mfdataset(
    smrf_path + '*.nc',
    parallel=True,
)

## Solar elevation 

In [ ]:
a = (90 - smeshr_solar.zenith)
b = (90 - smrf_solar.zenith)
a.where(a < 90).hvplot(color='coral', label='Skyfield').opts(**PLOT_OPTS) * b.where(b < 90).hvplot(color='lightskyblue', label='SMRF')

### Difference

In [ ]:
(smeshr_solar.isel(time=index).zenith - smrf_solar.zenith).hvplot(**PLOT_OPTS)

## Solar azimuth

In [ ]:
smeshr_solar.where(smeshr_solar.azimuth > -181).azimuth.hvplot(color='coral', label='Skyfield').opts(**PLOT_OPTS) * \
smrf_solar.where(smrf_solar.azimuth > -181).azimuth.hvplot(color='lightskyblue', label='SMRF')

### Difference 

In [ ]:
(smeshr_solar.isel(time=index).azimuth - smrf_solar.azimuth).hvplot(**PLOT_OPTS)